In [14]:
import pandas as pd
import numpy as np
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt
import seaborn as sns
import math
import os
import cv2 as cv
import imageio
from scipy import spatial
from tqdm import tqdm
#import torch

In [15]:
############ module I structure reading ##############
### currently support: all-atom/sirah2 pdb libary
def readpdb(fi='4cmq.pdb',libfile='lib/mol.csv',iscg=False):
    pdb=open(fi,'r')
    index=[]
    name=[]
    resname=[]
    chain=[]
    resid=[]
    x=[]
    y=[]
    z=[]
    weight=[]
    element=[]
    lines=pdb.readlines()
    weightdict={}
    lib=pd.read_csv(libfile)
    if iscg:
        for i in lib.index:
            weightdict[(lib.residue[i],lib.element[i])]=lib.electron[i]
    else:
        for i in lib.index:
            weightdict[lib.element[i]]=lib.electron[i]
    for line in lines:
        if line[0:4] == 'ATOM':
            index.append(int(line[6:11]))
            n=line[12:16]
            name.append(n.strip())
            resname.append(line[17:20].strip())
            resid.append(int(line[22:26]))
            x.append(float(line[30:38]))
            y.append(float(line[38:46]))
            z.append(float(line[46:54]))
            if len(line)<77 or line[76:78].strip()=='':
                element.append(n.strip()[0])
            else:
                element.append(line[76:78].strip())
            if iscg:
                weight.append(weightdict[(resname[-1],element[-1])])
            else:
                weight.append(weightdict[element[-1]])
#            charge.append(int(line[78:80]))
    data=pd.DataFrame({
        'series':index,
        'name':name,
        'resname':resname,
        'resid':resid,
        'x':x,
        'y':y,
        'z':z,
        'weight':weight,
        'element':element
    })
    return data

In [16]:
############ module 0 math ##############
#Tait–Bryan angles https://en.wikipedia.org/wiki/Conversion_between_quaternions_and_Euler_angles
#eular=np.asarray[phi,theta,psi]
def eular2mat(eular):
    se=np.sin(eular)
    ce=np.cos(eular)
    mat=np.asarray([[ ce[2]*ce[1]   ,-se[2]*ce[1]+ce[2]*se[1]*se[0]  , se[2]*se[0]+ce[2]*se[1]*ce[0]] ,
                    [ se[2]*ce[1]   , ce[2]*ce[1]+se[2]*se[1]*se[0]  ,-ce[2]*se[0]+se[2]*se[1]*ce[0]] ,
                    [       se[1]   ,                   ce[1]*se[0]  ,                   ce[1]*ce[0]] ])
    return mat

def read_quat(fi='lib/c48n9.quat'):
    pdb=open(fi,'r')
    lines=pdb.readlines()
    quats=[]
    for line in lines[4:]:
        quat=np.asarray([float(line[0:12]),
                         float(line[13:25]),
                         float(line[26:38]),
                         float(line[39:51])])
        quats.append(quat)
    return quats

def gen_quater(libgrid='std60.csv',ang=5):
    grids=pd.read_csv(libgrid)
    theta=math.pi/ang
    dtheta=math.pi/ang
    quaterlist=[np.array([1,0,0,0])]
    for i in range(0,ang):
        s=math.sin(theta/2)
        c=math.cos(theta/2)
        for j in grids.index:
            quaterlist.append(np.array([c,s*grids.i[j],s*grids.j[j],s*grids.k[j]]))
        theta+=dtheta
    return quaterlist

def gen_quater_d(libgrid='std60.csv',ang=5):
    grids=pd.read_csv(libgrid)
    quaterlist=[np.array([1,0,0,0])]
    s=math.sin(ang*np.pi/360)
    c=math.cos(ang*np.pi/360)
    for j in grids.index:
        quaterlist.append(np.array([c,s*grids.i[j],s*grids.j[j],s*grids.k[j]]))
    return quaterlist

def quater2mat(q):
    mat=np.array([[1-2*q[2]*q[2]-2*q[3]*q[3],2*(q[1]*q[2]-q[3]*q[0]),2*(q[1]*q[3]+q[2]*q[0])],
                  [2*(q[1]*q[2]+q[3]*q[0]),1-2*q[1]*q[1]-2*q[3]*q[3],2*(q[2]*q[3]-q[1]*q[0])],
                  [2*(q[1]*q[3]-q[2]*q[0]),2*(q[2]*q[3]+q[1]*q[0]),1-2*q[1]*q[1]-2*q[2]*q[2]]])
    return mat

def eular2quater(eular):
    se=np.sin(eular/2)
    ce=np.cos(eular/2)
    q=np.asarray([ce[0]*ce[1]*ce[2]+se[0]*se[1]*se[2],
                  se[0]*ce[1]*ce[2]-ce[0]*se[1]*se[2],
                  ce[0]*se[1]*ce[2]+se[0]*ce[1]*se[2],
                  ce[0]*ce[1]*se[2]-se[0]*se[1]*ce[2]])
    return q

def quater2eular(q):
    eular=np.asarray([np.arctan2(2*(q[0]*q[1]+q[2]*q[3],1-2*(q[1]*q[1]+q[2]*q[2]))),
                      np.arcsin(2*(q[0]*q[2]-q[1]*q[3])),
                      np.arctan2(2*(q[0]*q[3]+q[1]*q[2],1-2*(q[3]*q[3]+q[2]*q[2])))])
    return eular

In [17]:
############ module II generate libaray ##############
'''def pdb_trans(pdb,trans):
#transform define as (psi,theta,phi,dx,dy)
    trans_pdb=pdb.copy(deep=True)
    pos=np.matmul(np.array(data)[:,4:7],eular2mat(trans[:3]))
    pos[]
    trans_pdb[:,4:7]
    return trans_pdb'''

def proj(ori,resolution=3.7,libfile='lib/mol.csv',edge=3,iscg=0,drift=[0,0]):
    data=ori.copy()
    xmin=data.x.min()
    xmax=data.x.max()
    ymin=data.y.min()
    ymax=data.y.max()
    graph_mat=np.zeros((edge*2+math.ceil((xmax-xmin)/resolution),(edge*2+math.ceil((ymax-ymin)/resolution))))
    for i in data.index:
        graph_mat[int((data.x[i]-xmin)/resolution+edge)][int((data.y[i]-ymin)/resolution+edge)]+=data.weight[i]
    return graph_mat   

def projection(data=[],resolution=3.7,libfile='lib/mol.csv',edge=5,cutoff=1.5,diffu=3):
    xmin=data.x.min()-edge
    xmax=data.x.max()+edge
    ymin=data.y.min()-edge
    ymax=data.y.max()+edge
    graph_mat=np.zeros((math.ceil((xmax-xmin)/resolution),(math.ceil((ymax-ymin)/resolution))))
    lib=pd.read_csv(libfile)
    atomdict={}
    for i in lib.index:
        atomdict[lib.element[i]]=[lib.radius[i],lib.density[i]]
    for i in data.index:
        r=atomdict[data.element[i]][0]*cutoff*diffu/resolution               
        r2=atomdict[data.element[i]][0]*atomdict[data.element[i]][0]*diffu*diffu/(resolution*resolution)
        d=atomdict[data.element[i]][1]
        ax=(data.x[i]-xmin)/resolution
        ay=(data.y[i]-ymin)/resolution
        for x in range(int(ax-r),math.ceil(ax+r)):
            for y in range(int(ay-r),math.ceil(ay+r)):
                graph_mat[x][y]+=d*int(((ax-x)*(ax-x)+(ay-y)*(ay-y))<=r2)
    return graph_mat

def genlib(pdblib='pdblib',outdir='',
           quatfile='lib/c48u27.quat',edge=1,resolution=3.7,iscg=0,libmol='lib/mol.csv',size=[35,35],level=50):
    rot=read_quat(quatfile)
    lib={'pdb':[],'qr':[],'qi':[],'qj':[],'qk':[],'pic':[]}
    for pdb in os.listdir(pdblib):
        data=readpdb(pdblib+'/'+pdb,iscg=iscg,libfile=libmol)
        target=data.copy(deep=True)
        pos=np.array(data)[:,4:7]

        for i in tqdm(range(len(rot))):
            rotated=np.matmul(pos,quater2mat(rot[i]))[:,0:2]
            target.x=rotated[:,0]
            target.y=rotated[:,1]
            mat=proj(target,resolution=resolution,edge=edge)
            ori=[int((size[0]-mat.shape[0])/2),int((size[1]-mat.shape[1])/2)]
            lib['pdb'].append(pdb)
            lib['qr'].append(rot[i][0])
            lib['qi'].append(rot[i][1])
            lib['qj'].append(rot[i][2])
            lib['qk'].append(rot[i][3])
            pic=np.zeros(size)
            pic[ori[0]:ori[0]+mat.shape[0],ori[1]:ori[1]+mat.shape[1]]=mat
            pic=np.log(pic+1)
            pic=(pic-pic.mean()).astype('float32')
            lib['pic'].append(pic)
    lib=pd.DataFrame(lib)
    if outdir!='':
        os.mkdir(outdir)
        for i in lib.index:
            cv.imwrite(outdir+'/'+str(i)+'.png',lib.pic[i]*level)
    return lib


In [18]:
############ module I&III morphing ##############
def AdaptiveThresh(I,winSize,ratio=0.15):
    I_mean=cv.boxFilter(I,cv.CV_8U,winSize,borderType=cv.BORDER_REFLECT)
    out=I-(1.0-ratio)*I_mean
    out[out>=0]=255
    out[out<0]=0
    return out.astype(np.uint8)

def gen_Morph_parameter(lib,kernalsize_filter=(5,5),thresh=2.5,appsize=20,cviter=1):
    thresh_area=[999,0]
    thresh_dist=[10,0]
    circle=0
    liblist=[]
    for i in lib.index:
        bi_gray=np.zeros(lib.pic[i].shape)
        bi_gray[lib.pic[i]>thresh]=255
        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3))
        bi_gray = cv.morphologyEx(bi_gray, cv.MORPH_CLOSE, kernel,iterations=cviter)
        bi_gray = cv.morphologyEx(bi_gray, cv.MORPH_OPEN, kernel,iterations=cviter)
        contours, hierarchy = cv.findContours(bi_gray.astype('uint8'), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        largest=0
        for c, contour in enumerate(contours):
            Area = cv.contourArea(contour)
            if Area>=largest:
                largest=Area
                contour_r=contour.reshape(contour.size//2,2)
                maxdist=spatial.distance_matrix(contour_r, contour_r).max()
        liblist.append([largest,maxdist])

    liblist=np.asarray(liblist)
    thresh_contour = liblist[spatial.ConvexHull(liblist).vertices]
    return thresh_contour

def Extract_Protein(
    picdir,thresh_contour,
    outdir='test',
    cviter=3,
    picave=3,
    diiter=3,
    thresh_adt=0.35,
    thresh_pick=0.1,
    kernalsize_adt=(25,25),
    kernalsize_filter=(5,5),
    avekernal=[0.25,0.5,0.25],
    listthresh=30
    ):
    os.mkdir(outdir+'/binary')
    os.mkdir(outdir+'/ori')
#################################################################################
    extended_contour=np.zeros((thresh_contour.shape[0]+1,thresh_contour.shape[1]))
    extended_contour[:-1,:]=thresh_contour
    avekernal=np.asarray(avekernal)
    avecenter=avekernal.argmax()
    dirlist=os.listdir(picdir)
    pinserie=[]
    contourlist=[]
    print(thresh_contour)
    hullarea=spatial.ConvexHull(thresh_contour).volume
    for i,p in enumerate(dirlist):
        pinframe=[i+avecenter,dirlist[i+avecenter]]
        if i>=(len(dirlist)-picave):
            break
        ave=0
        ori=cv.imread(picdir+'/'+dirlist[i+avecenter],flags=cv.CV_8U)
        for j in range(picave):
            ave=ave+cv.imread(picdir+'/'+dirlist[i+j],flags=cv.CV_8U)*avekernal[j]
        burr=cv.GaussianBlur(ave.astype('uint8'), kernalsize_filter, 0)
        bi_gray= (255-AdaptiveThresh(burr,kernalsize_adt,thresh_adt)).astype('uint8')
        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3))
        bi_gray = cv.morphologyEx(bi_gray, cv.MORPH_OPEN, kernel,iterations=cviter)
        bi_gray = cv.morphologyEx(bi_gray, cv.MORPH_CLOSE, kernel,iterations=cviter)
        cv.imwrite(outdir+'/binary/'+dirlist[i+avecenter],bi_gray)
        contours, hierarchy = cv.findContours(bi_gray.astype('uint8'), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        outpic=ave.copy()

        pcounts=0
        for c, contour in enumerate(contours):

            Area = cv.contourArea(contour)
            if Area==0:
                continue

            contour_r=contour.reshape(contour.size//2,2)
            maxdist=spatial.distance_matrix(contour_r, contour_r).max()
            extended_contour[-1]=np.asarray([Area,maxdist])
            para_pick=spatial.ConvexHull(extended_contour).volume-hullarea
            if Area>=50:
                contourlist.append([p,c,Area,maxdist,contour.mean(axis=0)[0][0],contour.mean(axis=0)[0][1],para_pick])
            if para_pick<thresh_pick:
                cv.drawContours(ori,contours,c,255,1)
                pinframe.append([contour.mean(axis=0)[0],contour])
                pcounts+=1
        pinserie.append(pinframe)

        cv.imwrite(outdir+'/ori/'+dirlist[i+avecenter],ori)
    pd.DataFrame(contourlist,columns=['name','id','area','circle','x','y','para_dist']).to_csv(outdir+'/contour.csv')
    return pinserie
def Particle_clustering(pinserie,minobserve,maxmissing=15,maxdrift=7.0):
    plist=[]
    protein=[]
    while len(pinserie)>0:
        while len(pinserie[0])==2:
            pinserie.pop(0)
            if len(pinserie)==0:
                if len(protein)>=minobserve:
                    print('protein ',len(plist),'detected in ',len(protein),' frames')
                    plist.append(protein)
                return plist
        protein=[[pinserie[0][0],pinserie[0][1],pinserie[0][2][0],pinserie[0][2][1]]]
        print('protein',len(plist),' detected at frame ',protein[-1][0],' centered at ',protein[-1][2])
        pinserie[0].pop(2)
        missedframe=1
        for pl in pinserie[1:]:
            for pi in range(2,len(pl)):
                if ((protein[-1][2]-pl[pi][0])**2).sum() <missedframe*maxdrift*maxdrift:
                    protein.append([pl[0],pl[1],pl[pi][0],pl[pi][1]])
                    pl.pop(pi)
                    missedframe=0
                    break
            missedframe+=1
            if missedframe>=maxmissing:
                break
        if len(protein)>=minobserve:
            print('protein ',len(plist),'detected in ',len(protein),' frames')
            plist.append(protein)
    return plist
def Gen_picked_pic(plist,dirin,outdir,
                   kernel=cv.getStructuringElement(cv.MORPH_ELLIPSE,(5,5)),dilate_iter=3,picsize=[35,35],
                   debug=True):
    count=0
    dirlist=[]
    drifts=[]
    for i in plist:
        count+=1
        os.mkdir(outdir+'/protein_'+str(count))
        os.mkdir(outdir+'/ori_'+str(count))
        dirlist.append(outdir+'/protein_'+str(count))
        drift=[]
        for c in i:
            ori_pic=cv.imread(dirin+'/'+c[1],flags=cv.CV_8U)
            mask=np.zeros(ori_pic.shape,dtype='uint8')
            cv.drawContours(mask,(c[3],),0,255,cv.FILLED)
            mask=cv.dilate(mask,kernel=kernel,iterations=dilate_iter)
            masked=(mask/255*(255-ori_pic))
            masked[masked==0]=255-ori_pic.mean()
            pos=[int(c[2][0]),int(c[2][1])]
            cutted=masked[max(0,pos[1]-picsize[1]):min(ori_pic.shape[1],pos[1]+picsize[1]),max(0,pos[0]-picsize[0]):min(ori_pic.shape[0],pos[0]+picsize[0])].astype('uint8')
            cutted_ori=ori_pic[max(0,pos[1]-picsize[1]):min(ori_pic.shape[1],pos[1]+picsize[1]),max(0,pos[0]-picsize[0]):min(ori_pic.shape[0],pos[0]+picsize[0])].astype('uint8')
            cv.drawContours(ori_pic,(c[3],),0,255,1)
            drift.append((max(0,pos[1]-picsize[1]),max(0,pos[0]-picsize[0])))
            cv.imwrite(outdir+'/ori_'+str(count)+'/'+c[1],cutted_ori)
            cv.imwrite(outdir+'/protein_'+str(count)+'/'+c[1],cutted)
        drifts.append(drift)
    return dirlist,drifts
def average_pic(picdir='2',ave=5,outdir='pic_average'):
    dirlist=os.listdir(picdir)
    os.system('mkdir '+outdir)
    frames=[]
    summary=0
    for i in range(ave):
        frames.append(mpimg.imread(picdir+'/'+dirlist[i]))
        summary=frames[i]+summary
        mpimg.imsave(outdir+'/0.png',summary/ave)
    count=0
    for i in range(ave,len(dirlist)):
        if count>=ave:
            count=0
        frames[count]=mpimg.imread(picdir+'/'+dirlist[i])
        mpimg.imsave(outdir+'/'+str(i-ave+1)+'.png',summary/ave)
    return

In [19]:
def makegif(fits,picdir,outname='',duration=0.3):

    pics=os.listdir(picdir)
    if fits==0:
        oriframes=[]
        for i in pics:
            oriframes.append(cv.imread(picdir+'/'+i))
        print('origin gif saved as ori.gif')
        imageio.mimsave(outname+'_ori.gif', oriframes, 'GIF', duration=duration)
        return
    simframes=[]
    for i in pics:
        frame=cv.imread(picdir+'/'+i)
        if frame.ndim==1:
            frame=cv.cvtColor(frame,cv.COLOR_GRAY2RGB)
        if not i in fits:
            simframes.append(frame.copy())
            continue

        if type(fits[i][0])==int:
            simframes.append(frame.copy())
            continue
        ys=fits[i][2]-len(fits[i][0])
        xs=fits[i][1]-len(fits[i][0][0])
        fm=fits[i][0].max()
        for y in range(ys,fits[i][2]):
            for x in range(xs,fits[i][1]):

                t=255*fits[i][0][y-ys][x-xs]/fm

                frame[y,x,:]=t
        simframes.append(frame.copy())
    imageio.mimsave(outname+'_fitted.gif', simframes, 'GIF', duration=duration)
    return

In [20]:
def cvmatch(lib,pic,drift,ori=0,r=10,method=cv.TM_CCOEFF_NORMED):

    tempic=cv.imread(pic,cv.CV_8U).astype('float32')

    bestfit=[0,0,0,0,0,0]
    cvlib=lib
    scores=[]

    for p in cvlib.index:
        i=cvlib.pic[p]
        cmatch=cv.matchTemplate(tempic,i,method)
        m=cmatch.argmax()
        l=len(cmatch[0])

        score=cmatch.max()
        scores.append(score)

        if score>bestfit[3]:
            bestfit[0]=i
            bestfit[1]=m%l+i.shape[1]
            bestfit[2]=int(m//l)+i.shape[0]
            bestfit[3]=score
            bestfit[5]=cmatch
            bestfit[4]=cvlib.pdb[p]
    return bestfit,scores

In [21]:
def autosearch(picdir,template,
               outdir='fitted',debug=True,
               quatfile='lib/c48u27.quat',resolution=3.7,rf=10,pick=[],
               libmol='lib/mol.csv',iscg=0,picsize=[35,35],thresh_pick=0.01,
               minobserve=100,thresh_adt=0.35):
    dirlist=os.listdir(picdir)
    os.mkdir(outdir)
    if type(template)==str:
        lib=genlib(template,quatfile,resolution=resolution,iscg=iscg,libmol=libmol,size=[picsize])
        print('lib generated')
        thresh_contour=gen_Morph_parameter(lib)
    elif type(template)==pd.DataFrame:
        lib=template
        print('use input lib')
        picsize=lib.pic[0].shape
        thresh_contour=gen_Morph_parameter(lib)
    else:
        raise ValueError('input lib must be a folder contains pdb files or a libary generated by genlib')
    if len(pick)==3:
            thresh_contour=np.asarray([[pick[0]*pick[1]/(resolution),np.sqrt(pick[0]*pick[0]+pick[1]*pick[1])],
                                       [pick[2]*pick[1]/(resolution),np.sqrt(pick[2]*pick[2]+pick[1]*pick[1])],
                                       [pick[0]*pick[2]/(resolution),np.sqrt(pick[0]*pick[0]+pick[2]*pick[2])],])/resolution
            print('pick particles with length: ',template)
    print('pick range generated')
    pinserie=Extract_Protein(picdir,thresh_contour,outdir=outdir,thresh_adt=thresh_adt,thresh_pick=thresh_pick)
    print('contours found')    
    plist=Particle_clustering(pinserie,minobserve,maxmissing=30,maxdrift=7)
    print('particle clustered')
    picked_dirlist,drifts=Gen_picked_pic(plist,picdir,outdir=outdir,
                                         kernel=cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3)),dilate_iter=3,picsize=picsize)
    print('picked picture generated')
    outdata=[]
    for pc,protein in enumerate(picked_dirlist):
        fits={}
        dirlist_pro=os.listdir(protein)
        list_score=np.zeros([len(dirlist_pro),len(lib)])
        for c,i in enumerate(dirlist_pro):
            fits[i],list_score[c]=cvmatch(lib,protein+'/'+i,drifts[pc][c])
            print('protein '+protein+' in picture',i,'is fitted as',fits[i][4],'fitted score:',fits[i][3])
        makegif(fits,outdir+'/protein_'+str(pc+1),outdir+'/protein_'+str(pc+1)+'movie')
        print('gif for protein '+str(pc+1)+' saved as '+outdir+'/protein_'+str(pc+1)+'movie.gif')
        table_score=pd.DataFrame(list_score)
        table_score.index=dirlist_pro
        table_score.to_csv(outdir+'/protein_'+str(pc+1)+'.csv')
        outdata.append(fits)
        makegif(0,outdir+'/ori_'+str(pc+1),outdir+'/protein_'+str(pc+1)+'ori.gif')
    return outdata

def drawcmatch(pdblib='GAPRPDB',lib='',picdir='2',outname='sim.gif',ang=5,libgrid='std60.csv',
         resolution=3.7,rf=10,oriout='',libmol='lib/mol.csv',iscg=0,picaver=0):
    if picaver:
        os.system('mkdir TEMPPIC')
        average_pic(picdir,ave=picaver,outdir='TEMPPIC')
        picdir='TEMPPIC'
    if lib:
        print('use input lib')
    else:
        lib=genlib(pdblib,libgrid,ang,resolution=resolution,iscg=iscg,libmol=libmol)
        print('lib generated')
    cmatch=[]
    dirlist=os.listdir(picdir)
    for i in dirlist:
        cmatch.append(cvmatch(lib,picdir+'/'+i))
    print('gif saved as ',outname)
    return cmatch

In [47]:
libecori=genlib(pdblib='pdblib_4zt0', quatfile='lib/c48u27.quat', resolution=1.0, size=(256, 256), outdir='Cas9_4zt0.256', level=100)
fits_ecori=autosearch(picdir='Cas9_4zt0.256', template=libecori, outdir='Cas9_4zt0.256.fitWT', rf=10, minobserve=5, thresh_adt=0.35, debug=True)

  0%|          | 0/648 [00:00<?, ?it/s]

100%|██████████| 648/648 [01:25<00:00,  7.59it/s]


use input lib
pick range generated
[[ 650.5          49.81967483]
 [1261.           53.33854141]
 [3136.           85.79627032]
 [3287.5          94.81033699]
 [3044.          103.40696302]
 [2885.          106.32027088]
 [2632.          105.90561836]
 [2291.5         104.69479452]
 [1303.           88.52683209]
 [1030.5          81.15417426]]
contours found
particle clustered
picked picture generated


In [48]:
libecori=genlib(pdblib='pdblib_4cmq', quatfile='lib/c48u27.quat', resolution=1.0, size=(256,256), outdir='Cas9_4cmq.256', level=100)
fits_ecori=autosearch(picdir='Cas9_4cmq.256', template=libecori, outdir='Cas9_4cmq.256.fitWT', rf=10, minobserve=5, thresh_adt=0.35, debug=True)

100%|██████████| 648/648 [02:33<00:00,  4.23it/s]


use input lib
pick range generated
[[4454.          138.43771163]
 [3907.5         161.07762104]
 [2498.          133.18408313]
 [1738.5         113.52973179]
 [ 531.5          69.33974329]
 [ 426.5          61.22091146]
 [ 361.5          45.22167622]
 [ 344.5          34.71310992]
 [ 366.5          32.24903099]
 [ 399.           32.69556545]
 [3750.           88.20430828]
 [3963.           91.98369421]]
contours found
particle clustered
picked picture generated


In [49]:
libecori=genlib(pdblib='pdblib', quatfile='lib/c48u27.quat', resolution=1.0, size=(256,256), outdir='Cas9.256', level=100)
fits_ecori=autosearch(picdir='Cas9.256', template=libecori, outdir='Cas9.256.fitWT', rf=10, minobserve=5, thresh_adt=0.35, debug=True)

100%|██████████| 648/648 [02:33<00:00,  4.23it/s]


use input lib
pick range generated
[[4454.          138.43771163]
 [3907.5         161.07762104]
 [2498.          133.18408313]
 [1738.5         113.52973179]
 [ 531.5          69.33974329]
 [ 426.5          61.22091146]
 [ 361.5          45.22167622]
 [ 344.5          34.71310992]
 [ 366.5          32.24903099]
 [ 399.           32.69556545]
 [3750.           88.20430828]
 [3963.           91.98369421]]
contours found
particle clustered
picked picture generated
